In [3]:
import os
import json
from collections import Counter, defaultdict

# Ruta base donde buscar los archivos .json
base_dir = "../datasets/fallos_json"

# Diccionario para almacenar la información
result = {}

# Recorrer todas las carpetas y archivos .json
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".json"):
            file_path = os.path.join(root, file)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    data = json.load(f)
                    citations = (
                        data.get("METADATOS", {})
                        .get("ARTICULOS_CITADOS", {})
                        .get("citations", [])
                    )
                    for citation in citations:
                        main_source = citation.get("main_source")
                        cited_articles = citation.get("cited_articles", [])
                        if main_source is not None and cited_articles is not None:
                            if main_source not in result:
                                result[main_source] = []
                            result[main_source].extend(cited_articles)
            except Exception as e:
                print(f"Error en {file_path}: {e}")

# Construir el JSON de salida
output = {}
for source, articles in result.items():
    freq = Counter(articles)
    output[source] = {
        "cited_articles_frequency": dict(freq),
        "total_cited_articles": sum(freq.values())
    }

# Guardar el resultado en un archivo
with open("main_source_cited_articles_stats.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

print("Archivo 'main_source_cited_articles_stats.json' generado correctamente.")

Archivo 'main_source_cited_articles_stats.json' generado correctamente.


# Prueba del procesador enriquecido de fallos legales

Vamos a procesar los archivos JSON usando el nuevo procesador y mostrar algunos fragmentos enriquecidos.

In [2]:
from pathlib import Path
import json
import logging
from typing import Iterator, Set, Dict, Any
from pydantic import ValidationError
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any

logger = logging.getLogger(__name__)

class LegalParagraphEnriched(BaseModel):
    """Modelo enriquecido para párrafos legales con contexto adicional"""
    expediente: str = Field(..., description="Número de expediente")
    section: str = Field(..., description="Sección del documento")
    paragraph_id: int = Field(..., ge=0, description="ID del párrafo dentro de la sección")
    text: str = Field(..., min_length=1, description="Contenido del párrafo")
    path: str = Field(..., description="Ruta relativa del archivo fuente")
    idea_central: Optional[str] = Field(None, description="Idea central del fallo")
    articulos_citados: Optional[List[Dict[str, Any]]] = Field(None, description="Artículos citados en el fallo")
    materia_preliminar: Optional[str] = Field(None, description="Materia preliminar del fallo")
    metadatos: Optional[Dict[str, Any]] = Field(None, description="Metadatos adicionales del fallo")

    def to_search_dict(self) -> Dict[str, Any]:
        """Convierte a formato para búsqueda"""
        return {
            "expte": self.expediente,
            "section": self.section,
            "paragraph": self.text,
            "path": self.path,
            "paragraph_id": self.paragraph_id,
            "idea_central": self.idea_central,
            "articulos_citados": self.articulos_citados,
            "materia_preliminar": self.materia_preliminar,
            "metadatos": self.metadatos
        }


class EnrichedProcessor:
    """Procesador que extrae fragmentos enriquecidos de los fallos JSON"""
    def __init__(self):
        self.stats = {
            "files_processed": 0,
            "paragraphs_extracted": 0,
            "expedientes_found": set(),
            "errors": []
        }

    def process_directory(self, json_dir: Path) -> Iterator[LegalParagraphEnriched]:
        """Procesa todos los archivos JSON en el directorio y subdirectorios"""
        json_files = list(json_dir.rglob("*.json"))
        logger.info(f"📄 Encontrados {len(json_files)} archivos")
        for file_path in json_files:
            try:
                yield from self._process_file(file_path, json_dir)
                self.stats["files_processed"] += 1
            except Exception as e:
                error_msg = f"Error en {file_path.name}: {str(e)}"
                logger.error(error_msg)
                self.stats["errors"].append(error_msg)
                continue
        logger.info(f"✅ Procesamiento completado: {self.stats}")

    def _process_file(self, file_path: Path, base_dir: Path) -> Iterator[LegalParagraphEnriched]:
        content = file_path.read_text(encoding="utf-8")
        docs = json.loads(content)
        if not isinstance(docs, list):
            docs = [docs]
        for doc in docs:
            yield from self._process_document(doc, file_path, base_dir)

    def _process_document(self, doc: Dict[str, Any], file_path: Path, base_dir: Path) -> Iterator[LegalParagraphEnriched]:
        contenido = doc["CONTENIDO"]
        expte = self._extract_expediente(contenido["INICIO"][0])
        idea_central = doc.get("IDEA_CENTRAL")
        articulos_citados = doc.get("METADATOS", {}).get("ARTICULOS_CITADOS", {}).get("citations", [])
        materia_preliminar = doc.get("MATERIA_PRELIMINAR")
        metadatos = doc.get("METADATOS", {})
        self.stats["expedientes_found"].add(expte)
        for section, paragraphs in contenido.items():
            if not isinstance(paragraphs, list):
                continue
            for idx, text in enumerate(paragraphs):
                if not text or not isinstance(text, str) or len(text.strip()) < 10:
                    continue
                try:
                    paragraph = LegalParagraphEnriched(
                        expediente=expte,
                        section=section,
                        paragraph_id=idx,
                        text=text,
                        path=file_path.relative_to(base_dir).as_posix(),
                        idea_central=idea_central,
                        articulos_citados=articulos_citados,
                        materia_preliminar=materia_preliminar,
                        metadatos=metadatos
                    )
                    self.stats["paragraphs_extracted"] += 1
                    yield paragraph
                except ValidationError:
                    continue

    def _extract_expediente(self, header: str) -> str:
        for pattern in ["Expte. Nº", "Expediente", "Exp."]:
            if pattern in header:
                return header.split(pattern)[-1].strip().split()[0]
        import hashlib
        return hashlib.md5(header.encode()).hexdigest()[:8].upper()

    def get_stats(self) -> dict:
        return {
            "processor_type": "enriched",
            "files_processed": self.stats["files_processed"],
            "paragraphs_extracted": self.stats["paragraphs_extracted"],
            "expedientes_found": len(self.stats["expedientes_found"]),
            "errors": len(self.stats["errors"]),
            "error_rate": len(self.stats["errors"]) / max(1, self.stats["files_processed"])
        }


# Ruta al directorio de fallos JSON
json_dir = Path("datasets/fallos_json/2024/07")

# Instanciar el procesador enriquecido
processor = EnrichedProcessor()

# Procesar el directorio y obtener los fragmentos enriquecidos
fragmentos = list(processor.process_directory(json_dir))

print(f"Total de fragmentos extraídos: {len(fragmentos)}")

Total de fragmentos extraídos: 578


In [5]:
# Mostrar los primeros 3 fragmentos enriquecidos
for frag in fragmentos[:3]:
    print(f"Expte: {frag.expediente}")
    print(f"Sección: {frag.section}")
    print(f"Texto: {frag.text}")
    print(f"Idea central: {frag.idea_central[:100] if frag.idea_central else None}")
    print(f"Artículos citados: {frag.articulos_citados}")
    print(f"Materia: {frag.materia_preliminar}")
    print("-" * 60)

Expte: 8554
Sección: INICIO
Texto: "FIGUEROA MERCEDES NOEMI POR SI Y EN NOMBRE Y REPRESENTACIÓN DE SUS HIJOS MENORES DE EDAD C/ FONTANINI CRISTIAN MATIAS Y OTROS S/ ORDINARIO ACCIDENTES DE TRANSITO" - Expte. Nº 8554
Idea central: **IDEA CENTRAL DEL FALLO:**

El Tribunal declaró inadmisible el recurso de inaplicabilidad de ley in
Artículos citados: [{'main_source': 'Ley 24.449', 'cited_articles': [68], 'extra': None}, {'main_source': 'Ley 7046', 'cited_articles': [64, 3, 30, 63, 94], 'extra': 'modif. por ley 11.141'}, {'main_source': 'Código Procesal Civil y Comercial (CPCC)', 'cited_articles': [65, 277, 280], 'extra': None}, {'main_source': 'Acuerdo General 15/18 SNE', 'cited_articles': [1, 4], 'extra': None}, {'main_source': 'Acuerdo General 11/20', 'cited_articles': [0], 'extra': 'del 23-6-2020'}, {'main_source': 'Otro', 'cited_articles': [56, 118, 29, 28, 114], 'extra': 'párr. 3, de la Ley 17.148'}]
Materia: ORDINARIO ACCIDENTES DE TRANSITO
------------------------------------------

Si ves los campos extra correctamente poblados, ¡el procesador está funcionando bien!